In [8]:
library(dplyr)
library(readr)
library(lubridate)

Le chargement a nécessité le package : timechange


Attachement du package : ‘lubridate’


Les objets suivants sont masqués depuis ‘package:base’:

    date, intersect, setdiff, union




In [4]:
df <- read_csv("../../data/dataverse_files/Electricity_B1E.csv")

Rows: 1051200 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (12): unix_ts, V, I, f, DPF, APF, P, Pt, Q, Qt, S, St

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [78]:
df_dt <- df %>%
    mutate(
        unix_ts = as.POSIXct(unix_ts, origin = "1970-01-01 12:00:00"),
        date = as.Date(format(as.POSIXct(unix_ts, origin = "1970-01-01 12:00:00"), format = "%Y-%m-%d"), format = "%Y-%m-%d"),
        hour = format(as.POSIXct(unix_ts, origin = "1970-01-01 12:00:00"), format = "%H"),
        minute = format(as.POSIXct(unix_ts, origin = "1970-01-01 12:00:00"), format = "%M"),
        minuteHalf = case_when(
            as.integer(format(as.POSIXct(unix_ts, origin = "1970-01-01 12:00:00"), format = "%M")) <= 30 ~ "00",
            as.integer(format(as.POSIXct(unix_ts, origin = "1970-01-01 12:00:00"), format = "%M")) > 30 ~ "30"
        )
    ) 
    
# df_dt %>%
#     filter(unix_ts < "2012-04-01 17:00:00") %>%
#     group_by(hour, minute, Pt) %>%
#     summarise()

In [79]:
glimpse(df_dt)

Rows: 1,051,200
Columns: 16
$ unix_ts    <dttm> 2012-04-01 15:00:00, 2012-04-01 15:01:00, 2012-04-01 15:02…
$ V          <dbl> 117.8, 118.1, 118.1, 118.1, 117.7, 117.5, 117.6, 117.4, 117…
$ I          <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…
$ f          <dbl> 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60,…
$ DPF        <dbl> 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00,…
$ APF        <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00,…
$ P          <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…
$ Pt         <dbl> 1755, 1755, 1755, 1755, 1755, 1755, 1755, 1755, 1755, 1755,…
$ Q          <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…
$ Qt         <dbl> 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112,…
$ S          <dbl> 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,…
$ St         <dbl> 11329, 11330, 11330, 11330, 11330, 11330, 11330, 11330, 113…
$ date      

In [84]:
firstHourDF <- df_dt %>%
    filter(unix_ts < "2012-04-01 16:00:00")

firstBillingDF <- df_dt %>%
    group_by(date) %>%
    summarise(
        kwh = (sum(V * I))
    ) %>%
    filter(between(date, as.Date("2012-04-01", format = "%Y-%m-%d"), as.Date("2012-04-19", format = "%Y-%m-%d")))

In [88]:
df_dt %>%
    filter(between(date, as.Date("2012-04-01", format = "%Y-%m-%d"), as.Date("2012-04-19", format = "%Y-%m-%d"))) %>%
    summarise(
        kwh = (sum(120 * 15))
    )

kwh
<dbl>
1800


In [86]:
firstBillingDF %>%
    summarise(avgKwh = sum(kwh))

avgKwh
<dbl>
20469.1


In [7]:
df %>%
    filter(I != 0.0) %>%
    mutate(`V x I` = V * I)

unix_ts,V,I,f,DPF,APF,P,Pt,Q,Qt,S,St,V x I
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1333334280,117.5,0.2,60,1,0.91,21,1763,0,113,24,11502,23.50
1333334340,117.3,0.2,60,1,0.92,21,1763,0,113,23,11503,23.46
1333334400,117.1,0.2,60,1,0.89,21,1764,0,113,24,11503,23.42
1333334460,116.8,0.2,60,1,0.91,21,1764,0,113,24,11504,23.36
1333334520,116.7,0.2,60,1,0.89,21,1764,0,113,24,11504,23.34
1333334580,116.7,0.2,60,1,0.92,21,1765,0,113,23,11504,23.34
1333334640,117.2,0.1,60,1,0.90,21,1765,0,113,23,11505,11.72
1333334700,117.0,0.2,60,1,0.90,21,1765,0,113,23,11505,23.40
1333334760,117.2,0.1,60,1,0.92,21,1766,0,113,23,11506,11.72


In [5]:
df %>%
    summarize(
        test = sum(V * I)
    )

test
<dbl>
929256.9
